## Google Colab utils
## *Don't forget to set GPU*

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd "/gdrive/My Drive/air-pollution"

## Evaluating

In [1]:
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, Lambda, Reshape, Dropout
from tensorflow.keras.layers import Bidirectional, RepeatVector, Dot, Activation
from tensorflow.keras.layers import Concatenate, Flatten

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from datetime import datetime
from utils import *
import pickle
from collections import namedtuple


Bad key "text.kerning_factor" on line 4 in
/home/zafir/miniconda3/envs/tensorflow/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
generate_folder_structure()

In [3]:
sensor = 'Miladinovci'

In [4]:
PretrainedModels = namedtuple('PretrainedModels', ['standard', 'attentive'])
best_models = {
    'Centar': PretrainedModels('2020-06-06-08-53-36', '2020-06-06-08-56-06'),
    'Karpos': PretrainedModels('2020-06-06-09-06-42', '2020-06-06-09-08-54'),
    'Lisice': PretrainedModels('2020-06-06-09-17-17', '2020-06-06-09-18-28'),
    'Rektorat': PretrainedModels('2020-06-06-09-28-19', '2020-06-06-09-29-38'),
    'Miladinovci': PretrainedModels('2020-06-05-17-52-45', '2020-06-05-17-55-40'),
}

In [5]:
def retransform_data(scaler, data):
    data = scaler.inverse_transform(data)
    data = np.exp(data) - 1
    return data

In [6]:
with open(f'./pickles/scalers/{sensor}/PM10', 'rb') as f:
    scaler = pickle.load(f)

In [7]:
train_encoder_input_data = np.load(f'./data/third-order/{sensor}/train_encoder_input_data.npy')
train_decoder_input_data = np.load(f'./data/third-order/{sensor}/train_decoder_input_data.npy')
train_decoder_target_data = np.load(f'./data/third-order/{sensor}/train_decoder_target_data.npy')

valid_encoder_input_data = np.load(f'./data/third-order/{sensor}/valid_encoder_input_data.npy')
valid_decoder_input_data = np.load(f'./data/third-order/{sensor}/valid_decoder_input_data.npy')
valid_decoder_target_data = np.load(f'./data/third-order/{sensor}/valid_decoder_target_data.npy')

test_encoder_input_data = np.load(f'./data/third-order/{sensor}/test_encoder_input_data.npy')
test_decoder_input_data = np.load(f'./data/third-order/{sensor}/test_decoder_input_data.npy')
test_decoder_target_data = np.load(f'./data/third-order/{sensor}/test_decoder_target_data.npy')
test_decoder_target_data = test_decoder_target_data.flatten()
y_true_retransformed = retransform_data(scaler, test_decoder_target_data)

In [8]:
m_train, Tx, encoder_input_dim = train_encoder_input_data.shape
    
Ty, decoder_input_dim = (train_decoder_input_data.shape[1], 
                         train_decoder_input_data.shape[2])

decoder_output_dim = train_decoder_target_data.shape[2]

m_val = valid_encoder_input_data.shape[0]
m_test = test_decoder_input_data.shape[0]

## Standard Train Model

First, using the best hyperparameters found during the random search, we fine tune the model.

In [ ]:
latent_dim = 64
dense_dropout_rate = 0
learning_rate = 0.000219

In [ ]:
K.clear_session()

# ------------------- SHARED LAYERS ---------------------
encoder_lstm = LSTM(latent_dim, return_state=True, 
                      name='encoder_lstm')
decoder_lstm = LSTM(latent_dim, return_state=True, 
                    return_sequences=True, name='decoder_lstm')
decoder_dense = Dense(decoder_output_dim, 
                      activation='linear', name='decoder_dense')
dense_dropout = Dropout(rate=dense_dropout_rate, name='dense_dropout')

# -------------------- TRAIN MODEL ----------------------
encoder_inputs = Input(shape=(Tx, encoder_input_dim), 
                       name='encoder_inputs')

decoder_inputs = Input(shape=(Ty, decoder_input_dim), 
                       name='decoder_inputs')

# Obtain the hidden states of the encoder
_, h, c = encoder_lstm(encoder_inputs)

# Obtain the outputs of the decoder (we disregard the hidden 
# states during training)
x, _, _ = decoder_lstm(decoder_inputs, initial_state=[h, c])
x = dense_dropout(x)
decoder_outputs = decoder_dense(x)

model = Model(inputs=[encoder_inputs, decoder_inputs], 
              outputs=decoder_outputs)
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='mse')

In [ ]:
timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
print(f'Starting training at: {timestamp}')

history = model.fit(x=[train_encoder_input_data, 
                       train_decoder_input_data], 
                    y=train_decoder_target_data,
                    validation_data=([
                       valid_encoder_input_data,
                       valid_decoder_input_data],
                       valid_decoder_target_data),
                    batch_size=64,
                    epochs=100,
                    callbacks=[ModelCheckpoint(f'./checkpoints/{sensor}/standard/{timestamp}.h5', 
                                               save_best_only=True),
                               EarlyStopping(monitor='val_loss', 
                                             patience=10, 
                                             verbose=1)])

with open(f'./histories/{sensor}/standard/{timestamp}.pkl', 'wb') as output:
    pickle.dump(history.history, output)
    
with open(f'./histories/{sensor}/standard/results.csv', 'a') as file:
    min_epoch = np.argmin(history.history['val_loss'])
    file.write(f"{timestamp}, {min_epoch}, {history.history['loss'][min_epoch]}, {history.history['val_loss'][min_epoch]}\n")

In [ ]:
with open(f'./histories/{sensor}/standard/{timestamp}.pkl', 'rb') as inp:
    history = pickle.load(inp)

plt.figure(figsize=(15, 7))
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

## Standard Inference Model

Now, load the best (trained) model, obtain the layers and build an inference model

In [9]:
timestamp = best_models[sensor].standard
best_model = load_model(f'./checkpoints/{sensor}/standard/{timestamp}.h5')

encoder_lstm = best_model.get_layer('encoder_lstm')
decoder_lstm = best_model.get_layer('decoder_lstm')
dense_dropout = best_model.get_layer('dense_dropout')
decoder_dense = best_model.get_layer('decoder_dense')

In [10]:
K.clear_session()

# ------------------ INFERENCE MODEL --------------------
encoder_inputs = Input(shape=(Tx, encoder_input_dim), 
                       name='encoder_inputs')

decoder_inputs = Input(shape=(Ty, decoder_input_dim), 
                       name='decoder_inputs')

# Obtain the hidden states of the encoder
_, h, c = encoder_lstm(encoder_inputs)

outputs = []

for t in range(Ty):
    if t == 0:
        x = Lambda(lambda z: z[:, t, :])(decoder_inputs)
    else:
        x = Lambda(lambda z: z[:, t, 1:])(decoder_inputs)
        x = Concatenate(axis=-1)([out, x])
    
    decoder_input = Reshape((1, x.shape[1]))(x)
    
    # Obtain the output and hidden states of the decoder LSTM 
    out, h, c = decoder_lstm(decoder_input, initial_state=[h, c])
    out = dense_dropout(out)
    out = decoder_dense(out)
    out = Flatten()(out)
    outputs.append(out)

inference_model = Model(inputs=[encoder_inputs, decoder_inputs], 
              outputs=outputs, name='inference_model')

In [11]:
y_pred = inference_model.predict([test_encoder_input_data, test_decoder_input_data])
y_pred = format_model_output(y_pred)
y_pred_retransformed = retransform_data(scaler, y_pred)

In [12]:
rmse = np.sqrt(K.eval(tf.keras.losses.mean_squared_error(y_true_retransformed, 
                                                         y_pred_retransformed)))
r2 = r2_score(y_true_retransformed, y_pred_retransformed)
print(f'{sensor}\tArchitecture: standard\tRMSE: {rmse}\tR^2: {r2}')

Miladinovci	Architecture: standard	RMSE: 40.64072036743164	R^2: 0.23855493910746906


## Attentive Train Model

First, using the best hyperparameters found during the random search, we fine tune the model.

In [13]:
def one_step_attention(encoder_outputs, h_prev, attention_repeat, 
                       attention_concatenate, attention_dense_1,
                       attention_dense_2, attention_activation,
                       attention_dot):
    
    x = attention_repeat(h_prev)
    x = attention_concatenate([encoder_outputs, x])
    x = attention_dense_1(x)
    energies = attention_dense_2(x)
    alphas = attention_activation(energies)
    context = attention_dot([alphas, encoder_outputs])
    
    return context

In [14]:
def create_attentive_model(encoder_latent_dim, decoder_latent_dim,
                           attention_dense_dim, seq_dropout_rate,
                           dense_dropout_rate, learning_rate):
    K.clear_session()

    # ------------------- SHARED LAYERS ---------------------
    # Encoder layers
    encoder_lstm = Bidirectional(LSTM(encoder_latent_dim, return_sequences=True, 
                                      name='encoder_lstm'), merge_mode='concat')

    # Attention layers
    attention_repeat = RepeatVector(n=Tx, name='attention_repeat')
    attention_concatenate = Concatenate(axis=-1, name='attention_concatenate')
    attention_dense_1 = Dense(attention_dense_dim, activation='tanh', 
                              name='attention_dense_1')
    attention_dense_2 = Dense(1, activation='relu', name='attention_dense_2')
    attention_activation = Activation(softmax, name='attention_activation') 
    attention_dot = Dot(axes=1, name='attention_dot')

    # Decoder layers
    decoder_concatenate = Concatenate(axis=-1, name='decoder_concatenate')
    decoder_lstm = LSTM(decoder_latent_dim, return_state=True, 
                        name='decoder_lstm')
    decoder_dense = Dense(decoder_output_dim, activation='linear',
                          name='decoder_dense')

    seq_dropout = Dropout(rate=seq_dropout_rate, name='seq_dropout')
    dense_dropout = Dropout(rate=dense_dropout_rate, name='dense_dropout')

    # -------------------- TRAIN MODEL ----------------------
    encoder_inputs = Input(shape=(Tx, encoder_input_dim), 
                           name='encoder_inputs')
    
    decoder_inputs = Input(shape=(Ty, decoder_input_dim), 
                           name='decoder_inputs')

    x = encoder_lstm(encoder_inputs)
    encoder_outputs = seq_dropout(x)

    h0 = Input(shape=(decoder_latent_dim,), name='h0')
    c0 = Input(shape=(decoder_latent_dim,), name='c0')
    h, c = h0, c0

    # Decoder outputs
    outputs = []

    for t in range(Ty):
        context = one_step_attention(encoder_outputs, h, attention_repeat, 
                                     attention_concatenate, attention_dense_1,
                                     attention_dense_2, attention_activation,
                                     attention_dot)

        # Obtain the decoder input at timestamp t
        x = Lambda(lambda z: z[:, t, :])(decoder_inputs)
        decoder_input = Reshape((1, x.shape[1]))(x)

        # Construct the full decoder input by concatenating the input at 
        # timestemp t with the calculated context
        full_decoder_input = decoder_concatenate([decoder_input, context])

        h, _, c = decoder_lstm(full_decoder_input, initial_state=[h, c])
        x = dense_dropout(h)
        decoder_output = decoder_dense(x)

        outputs.append(decoder_output)

    model = Model(inputs=[encoder_inputs, decoder_inputs, h0, c0], 
                  outputs=outputs)
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')
    
    return model

In [15]:
encoder_latent_dim = 64
decoder_latent_dim = 96
attention_dense_dim = 12
seq_dropout_rate = 0.4
dense_dropout_rate = 0.2
learning_rate = 0.002

In [16]:
h0_train = np.zeros((m_train, decoder_latent_dim))
c0_train = np.zeros((m_train, decoder_latent_dim))

h0_val = np.zeros((m_val, decoder_latent_dim))
c0_val = np.zeros((m_val, decoder_latent_dim))

h0_test = np.zeros((m_test, decoder_latent_dim))
c0_test = np.zeros((m_test, decoder_latent_dim))

# due to the model architecture, we need to transform the output shape and type
train_attentive_decoder_target_data = list(np.swapaxes(
                                              train_decoder_target_data, 0, 1))
valid_attentive_decoder_target_data = list(np.swapaxes(
                                              valid_decoder_target_data, 0, 1))

In [ ]:
model = create_attentive_model(encoder_latent_dim, decoder_latent_dim,
                               attention_dense_dim, seq_dropout_rate,
                               dense_dropout_rate, learning_rate)

In [ ]:
timestamp = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
print(f'Starting training at: {timestamp}')

history = model.fit(x=[train_encoder_input_data, 
                       train_decoder_input_data,
                       h0_train, c0_train], 
                    y=train_attentive_decoder_target_data,
                    validation_data=([
                       valid_encoder_input_data,
                       valid_decoder_input_data, 
                       h0_val, c0_val],
                       valid_attentive_decoder_target_data),
                    batch_size=64,
                    epochs=100,
                    verbose=0,
                    callbacks=[LossPrintingCallback(Ty),
                               ModelCheckpoint(f'./checkpoints/{sensor}/attentive/{timestamp}/cpt',
                                               save_weights_only=True,
                                               save_best_only=True),
                               EarlyStopping(monitor='val_loss', 
                                             patience=10, 
                                             verbose=1)])

with open(f'./histories/{sensor}/attentive/{timestamp}.pkl', 'wb') as output:
    pickle.dump(history.history, output)
    
with open(f'./histories/{sensor}/attentive/results.csv', 'a') as file:
    min_epoch = np.argmin(history.history['val_loss'])
    file.write(f"{timestamp}, {min_epoch}, {history.history['loss'][min_epoch]}, {history.history['val_loss'][min_epoch]}\n")

In [ ]:
with open(f'./histories/{sensor}/attentive/{timestamp}.pkl', 'rb') as inp:
    history = pickle.load(inp)

plt.figure(figsize=(15, 7))
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()

## Attentive Inference Model

Now, load the best (trained) model, obtain the layers and build an inference model

In [17]:
timestamp = best_models[sensor].attentive

best_model = create_attentive_model(encoder_latent_dim, decoder_latent_dim,
                               attention_dense_dim, seq_dropout_rate,
                               dense_dropout_rate, learning_rate)

best_model.load_weights(f'./checkpoints/{sensor}/attentive/{timestamp}/cpt')

# ----------------- PRETRAINED LAYERS -------------------
# Encoder layers
encoder_lstm = best_model.get_layer('bidirectional')

# Attention layers
attention_repeat = best_model.get_layer('attention_repeat')
attention_concatenate = best_model.get_layer('attention_concatenate')
attention_dense_1 = best_model.get_layer('attention_dense_1')
attention_dense_2 = best_model.get_layer('attention_dense_2')
attention_activation = best_model.get_layer('attention_activation')
attention_dot = best_model.get_layer('attention_dot')

# Decoder layers
decoder_concatenate = best_model.get_layer('decoder_concatenate')
decoder_lstm = best_model.get_layer('decoder_lstm')
decoder_dense = best_model.get_layer('decoder_dense')

seq_dropout = best_model.get_layer('seq_dropout')
dense_dropout = best_model.get_layer('dense_dropout')

In [18]:
K.clear_session()

# ------------------ INFERENCE MODEL --------------------
encoder_inputs = Input(shape=(Tx, encoder_input_dim), 
                       name='encoder_inputs')

decoder_inputs = Input(shape=(Ty, decoder_input_dim), 
                       name='decoder_inputs')

x = encoder_lstm(encoder_inputs)
encoder_outputs = seq_dropout(x)

h0 = Input(shape=(decoder_latent_dim,), name='h0')
c0 = Input(shape=(decoder_latent_dim,), name='c0')
h, c = h0, c0

# Decoder outputs
outputs = []

for t in range(Ty):
    context = one_step_attention(encoder_outputs, h, attention_repeat, 
                                 attention_concatenate, attention_dense_1,
                                 attention_dense_2, attention_activation,
                                 attention_dot)
    
    # Obtain the decoder input at timestamp t. If it's the first timestep,
    # consider all of the features, otherwise use the calculated PM value
    # in the previous timestep.
    if t == 0:
        x = Lambda(lambda z: z[:, t, :])(decoder_inputs)
    else:
        x = Lambda(lambda z: z[:, t, 1:])(decoder_inputs)
        x = Concatenate(axis=-1)([decoder_output, x])
    
    decoder_input = Reshape((1, x.shape[1]))(x)

    # Construct the full decoder input by concatenating the input at 
    # timestemp t with the calculated context
    full_decoder_input = decoder_concatenate([decoder_input, context])

    h, _, c = decoder_lstm(full_decoder_input, initial_state=[h, c])
    x = dense_dropout(h)
    decoder_output = decoder_dense(x)
    outputs.append(decoder_output)

inference_model = Model(inputs=[encoder_inputs, decoder_inputs, h0, c0], 
                        outputs=outputs)

In [19]:
y_pred = inference_model.predict([test_encoder_input_data, test_decoder_input_data,
                                  h0_test, c0_test])
y_pred = format_model_output(y_pred)
y_pred_retransformed = retransform_data(scaler, y_pred)

In [20]:
rmse = np.sqrt(K.eval(tf.keras.losses.mean_squared_error(y_true_retransformed, 
                                                         y_pred_retransformed)))
r2 = r2_score(y_true_retransformed, y_pred_retransformed)
print(f'{sensor}\tArchitecture: attentive\tRMSE: {rmse}\tR^2: {r2}')

Miladinovci	Architecture: attentive	RMSE: 32.90631866455078	R^2: 0.5008003717648755
